In [3]:
from bs4 import BeautifulSoup

def getPotintsCoordinates(fileName):
    with open(fileName, 'r') as file:
        html_doc = file.read()
        soup = BeautifulSoup(html_doc, 'html.parser')
        points = [{"id": circle['id'], "x_coordinate": float(circle['cx']), "y_coordinate": float(circle['cy']) } for circle in soup.find_all('circle') + soup.find_all('ellipse')]
        return points 

In [8]:
pontosAlfa1Terreo = getPotintsCoordinates('Alfa1/Alfa1_terreo_sem_imagem.svg')
pontosAlfa1Andar1 = getPotintsCoordinates('Alfa1/Alfa1_1andar_sem_imagem.svg')
pontosAlfa1Andar2 = getPotintsCoordinates('Alfa1/Alfa1_2andar_sem_imagem.svg')
pontosAlfa1Andar3 = getPotintsCoordinates('Alfa1/Alfa1_3andar_sem_imagem.svg')

In [5]:
def insereIndexCsv(lista, inicio):
    for i in range(len(lista)):
        lista[i]['index_csv'] = i + inicio

In [10]:
insereIndexCsv(pontosAlfa1Terreo, 81)
insereIndexCsv(pontosAlfa1Andar1, 81+len(pontosAlfa1Terreo))
insereIndexCsv(pontosAlfa1Andar2, 81+len(pontosAlfa1Terreo)+len(pontosAlfa1Andar1))
insereIndexCsv(pontosAlfa1Andar3, 81+len(pontosAlfa1Terreo)+len(pontosAlfa1Andar1)+len(pontosAlfa1Andar2))

In [11]:
ref_1_terreo = "escada_alfa1_esquerda_terreo"
ref_2_terreo = "escada_alfa1_direita_terreo"
ref_1_andar_1 = "sala_de_aula_alfa1_104" 
ref_2_andar_1 = "sala_de_aula_alfa1_105"
ref_1_andar_2 = "sala_de_aula_alfa1_202" 
ref_2_andar_2 = "sala_de_aula_alfa1_203"
ref_1_andar_3 = "laboratorio_alfa1_302" 
ref_2_andar_3 = "laboratorio_alfa1_303"
escalaRefTerreo =  24.67
escalaRefAndar1 = 7.08
escalaRefAndar2 = 14.28
escalaRefAndar3 = 25.76

In [12]:
import numpy as np

def getDistances(array, ref1, ref2, scale):
    indexRefer1 = array.index(list(filter(lambda x: x['id'] == ref1, array))[0])
    indexRefer2 = array.index(list(filter(lambda x: x['id'] == ref2, array))[0])
    npAr = np.array([[val["x_coordinate"], val["y_coordinate"]] for val in array])
    absDistances = np.linalg.norm(npAr - npAr[:,None], axis=-1)
    distCart = absDistances[indexRefer1][indexRefer2]
    realDistances = (scale/distCart)*absDistances
    return realDistances

In [13]:
distanciasTerreo = getDistances(pontosAlfa1Terreo, ref_1_terreo, ref_2_terreo, escalaRefTerreo)
distanciasAndar1 = getDistances(pontosAlfa1Andar1, ref_1_andar_1, ref_2_andar_1, escalaRefAndar1)
distanciasAndar2 = getDistances(pontosAlfa1Andar2, ref_1_andar_2, ref_2_andar_2, escalaRefAndar2)
distanciasAndar3 = getDistances(pontosAlfa1Andar3, ref_1_andar_3, ref_2_andar_3, escalaRefAndar3)

In [14]:
def criaStringCsv(lista):
    for i in range(len(lista)):
       print('{},{},,'.format(lista[i]['index_csv'], lista[i]['id']))

In [15]:
criaStringCsv(pontosAlfa1Terreo+pontosAlfa1Andar1+pontosAlfa1Andar2+pontosAlfa1Andar3)

81,banheiro_alfa1_esquerda_terreo,,
82,cantina_alfa1,,
83,laboratorio_1_alfa1,,
84,escada_alfa1_esquerda_terreo,,
85,xerox,,
86,escada_alfa1_direita_terreo,,
87,laboratorio_2_alfa1,,
88,secretaria_academica,,
89,banheiro_alfa1_direita_terreo,,
90,refeitorio_alfa1,,
91,entrada_alfa1_1,,
92,entrada_alfa1_3,,
93,laboratorio_alfa1_101,,
94,sala_de_aula_alfa1_102,,
95,escada_alfa1_esquerda_1a,,
96,escada_alfa1_direita_1a,,
97,sala_de_aula_alfa1_101,,
98,sala_de_aula_alfa1_103,,
99,sala_de_estudos_alfa1_101,,
100,sala_de_aula_alfa1_104,,
101,sala_de_aula_alfa1_105,,
102,sala_de_aula_alfa1_106,,
103,banheiro_alfa1_direita_1a,,
104,laboratorio_alfa1_102,,
105,banheiro_alfa1_esquerda_1a,,
106,sala_de_aula_alfa1_202,,
107,banheiro_alfa1_direita_2a,,
108,sala_de_apoio_alfa1_2a,,
109,sala_de_estudos_alfa1_2a,,
110,escada_alfa1_direita_2a,,
111,escada_alfa1_esquerda_2a,,
112,banheiro_alfa1_esquerda_2a,,
113,sala_de_aula_alfa1_201,,
114,sala_de_aula_alfa1_204,,
115,sala_de_aula_alfa1_203,,
116,sala_

In [16]:
cutoff = 20
def criaAtributoDistanciaParaCsv(listaPontos, listaDistancias):
    distancias = {}
    begin = 0
    for i in range(len(listaDistancias)):
        distancias[listaPontos[i]['index_csv']] = {}
        for j in range(begin, len(listaDistancias[i])):
            if (listaDistancias[i][j] <= cutoff) and listaDistancias[i][j] > 0:
                distancias[listaPontos[i]['index_csv']][listaPontos[j]['index_csv']] = listaDistancias[i][j]
        begin += 1
        if distancias[listaPontos[i]['index_csv']] == {}:
            del distancias[listaPontos[i]['index_csv']]
    return distancias

In [17]:
dict_terreo = criaAtributoDistanciaParaCsv(pontosAlfa1Terreo, distanciasTerreo)
dict_andar1 = criaAtributoDistanciaParaCsv(pontosAlfa1Andar1, distanciasAndar1)
dict_andar2 = criaAtributoDistanciaParaCsv(pontosAlfa1Andar2, distanciasAndar2)
dict_andar3 = criaAtributoDistanciaParaCsv(pontosAlfa1Andar3, distanciasAndar3)

In [18]:
def criaStringEdgesCsv(dicionario, indexInicial):
    indexEdge = indexInicial
    for keyPrimaria in dicionario:
        for keyInterna in dicionario[keyPrimaria]:
            print('{},{},Undirected,{},,,{}'.format(keyPrimaria, keyInterna, round(dicionario[keyPrimaria][keyInterna], 1), indexEdge))
            indexEdge += 1
    return indexEdge

In [20]:
index_0 = criaStringEdgesCsv(dict_terreo, 230)
index_1 = criaStringEdgesCsv(dict_andar1, index_0)
index_2 = criaStringEdgesCsv(dict_andar2, index_1)
index_3 = criaStringEdgesCsv(dict_andar3, index_2)

81,83,Undirected,19.8,,,230
81,90,Undirected,13.9,,,231
82,83,Undirected,9.2,,,232
82,84,Undirected,8.8,,,233
82,90,Undirected,8.3,,,234
82,91,Undirected,3.2,,,235
82,92,Undirected,19.0,,,236
83,84,Undirected,1.7,,,237
83,90,Undirected,8.1,,,238
83,91,Undirected,10.5,,,239
83,92,Undirected,14.7,,,240
84,90,Undirected,9.3,,,241
84,91,Undirected,9.6,,,242
84,92,Undirected,13.3,,,243
85,86,Undirected,1.9,,,244
85,87,Undirected,18.3,,,245
85,88,Undirected,3.6,,,246
85,92,Undirected,12.0,,,247
86,87,Undirected,16.4,,,248
86,88,Undirected,2.8,,,249
86,92,Undirected,13.7,,,250
87,88,Undirected,16.3,,,251
87,89,Undirected,8.9,,,252
88,92,Undirected,15.4,,,253
90,91,Undirected,11.3,,,254
91,92,Undirected,17.4,,,255
93,94,Undirected,2.7,,,256
93,95,Undirected,2.0,,,257
93,97,Undirected,7.0,,,258
93,98,Undirected,8.2,,,259
93,99,Undirected,15.1,,,260
93,105,Undirected,19.7,,,261
94,95,Undirected,2.4,,,262
94,97,Undirected,7.3,,,263
94,98,Undirected,7.1,,,264
94,99,Undirected,14.2,,,265
95,97,Undi